In [1]:
import torch
from torch import nn,optim
from torch.nn import functional as F

In [6]:
class Residual(nn.Module):
    def __init__(self,input_channels,num_channles,use_1x1conv=False,strides=1):
        super().__init__()
        self.conv1=nn.Conv2d(input_channels,num_channles,kernel_size=3,padding=1,stride=strides)
        self.conv2=nn.Conv2d(num_channles,num_channles,kernel_size=3,padding=1)
        if use_1x1conv:
            self.conv3=nn.Conv2d(input_channels,num_channles,kernel_size=1,stride=strides)
        else:
            self.conv3=None
        self.bn1=nn.BatchNorm2d(num_channles)
        self.bn2=nn.BatchNorm2d(num_channles)
    
    def forward(self,X):
        Y=F.relu(self.bn1(self.conv1(X)))
        Y=self.bn2(self.conv2(Y))
        if self.conv3:
            X=self.conv3(X)
        Y+=X
        return F.relu(Y)

In [7]:
blk=Residual(3,3)
X=torch.rand(4,3,6,6)
Y=blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

In [8]:
blk=Residual(3,6,True,2)
blk(X).shape

torch.Size([4, 6, 3, 3])

In [9]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
nn.BatchNorm2d(64), nn.ReLU(),
nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [31]:
(224-7+3*2+2)/2

112.5

In [10]:
def resnet_block(input_channels,num_channels,num_residuals,first_block=False):
    blk=[]
    for i in range(num_residuals):
        if i ==0 and not first_block:
            blk.append(Residual(input_channels,num_channels,use_1x1conv=True,strides=2))
        else:
            blk.append(Residual(num_channels,num_channels))
    return blk

In [11]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [12]:
net=nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d((1,1)),nn.Flatten(),nn.Linear(512,10))

In [13]:
X=torch.rand(size=(1,1,224,224))

In [14]:
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,' output shape : ',X.shape)

Sequential  output shape :  torch.Size([1, 64, 56, 56])
Sequential  output shape :  torch.Size([1, 64, 56, 56])
Sequential  output shape :  torch.Size([1, 128, 28, 28])
Sequential  output shape :  torch.Size([1, 256, 14, 14])
Sequential  output shape :  torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d  output shape :  torch.Size([1, 512, 1, 1])
Flatten  output shape :  torch.Size([1, 512])
Linear  output shape :  torch.Size([1, 10])


In [15]:
def resnet18(num_classes, in_channels=1):
    """稍加修改的ResNet-18模型
    Defined in :numref:`sec_multi_gpu_concise`"""
    def resnet_block(in_channels, out_channels, num_residuals,
                     first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(in_channels, out_channels,
                                        use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(out_channels, out_channels))
        return nn.Sequential(*blk)

    # 该模型使用了更小的卷积核、步长和填充，而且删除了最大汇聚层
    net = nn.Sequential(
        nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU())
    net.add_module("resnet_block1", resnet_block(
        64, 64, 2, first_block=True))
    net.add_module("resnet_block2", resnet_block(64, 128, 2))
    net.add_module("resnet_block3", resnet_block(128, 256, 2))
    net.add_module("resnet_block4", resnet_block(256, 512, 2))
    net.add_module("global_avg_pool", nn.AdaptiveAvgPool2d((1,1)))
    net.add_module("fc", nn.Sequential(nn.Flatten(),
                                       nn.Linear(512, num_classes)))
    return net

In [16]:
net2=resnet18(10,1)

In [19]:
X

tensor([[ 0.4928,  0.3386,  0.1053, -0.4903,  0.3271, -0.0729, -0.2828, -0.1601,
         -0.3943, -0.4381]], grad_fn=<AddmmBackward0>)

In [20]:
X=torch.rand(size=(1,1,224,224))

In [21]:
X.shape

torch.Size([1, 1, 224, 224])

In [24]:
X=torch.rand(size=(1,1,224,224))

In [25]:
for layer in net2:
    X=layer(X)
    print(layer.__class__.__name__,' output shape : ',X.shape)

Conv2d  output shape :  torch.Size([1, 64, 224, 224])
BatchNorm2d  output shape :  torch.Size([1, 64, 224, 224])
ReLU  output shape :  torch.Size([1, 64, 224, 224])
Sequential  output shape :  torch.Size([1, 64, 224, 224])
Sequential  output shape :  torch.Size([1, 128, 112, 112])
Sequential  output shape :  torch.Size([1, 256, 56, 56])
Sequential  output shape :  torch.Size([1, 512, 28, 28])
AdaptiveAvgPool2d  output shape :  torch.Size([1, 512, 1, 1])
Sequential  output shape :  torch.Size([1, 10])


In [28]:
X=torch.rand(size=(1,1,224,224))

In [29]:
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,' output shape : ',X.shape)

Sequential  output shape :  torch.Size([1, 64, 56, 56])
Sequential  output shape :  torch.Size([1, 64, 56, 56])
Sequential  output shape :  torch.Size([1, 128, 28, 28])
Sequential  output shape :  torch.Size([1, 256, 14, 14])
Sequential  output shape :  torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d  output shape :  torch.Size([1, 512, 1, 1])
Flatten  output shape :  torch.Size([1, 512])
Linear  output shape :  torch.Size([1, 10])
